In [10]:
config = {
    "base_dir":        "/storage/vbsjjlnu/VBSjjlnu_v7/",
    "plot_config":     "FullRun2_v7",
    "cut":             "res_sig",
    "model_version":   "v5",
}


import os

config_base_dir = os.path.join(config["base_dir"], config["plot_config"])

# create the model directory
model_dir   = os.path.join(config_base_dir, config["cut"] , "models",  config["model_version"])
os.makedirs(model_dir, exist_ok=True)

import yaml
model_config_file = open(model_dir + "/model_config.yml", "r")
model_config = yaml.safe_load(model_config_file)

for key in ["samples_version", "cols"]:
    config[key] = model_config[key]

In [11]:
import tensorflow as tf


# for tf v2, fgo into v1 compatibility mode
if tf.__version__.startswith("2."):
    tf = tf.compat.v1
tf.disable_eager_execution()

sess = tf.Session()

tf.keras.backend.set_session(sess)


model = tf.keras.models.load_model(os.path.join(model_dir, "model.h5"))


#————- Salvo qui ————
# inputs:  ['Input']
print('inputs: ', [input.op.name for input in model.inputs])

# outputs:  ['Output/Sigmoid']
print('outputs: ', [output.op.name for output in model.outputs])

outputs = [output.op.name for output in model.outputs]
constant_graph = tf.graph_util.convert_variables_to_constants(
    sess, sess.graph.as_graph_def(), outputs)
tf.train.write_graph(constant_graph, model_dir, "model.pb", as_text=False)

## save tensorflow metadata
with open(os.path.join(model_dir, "tf_metadata.txt"), "w") as f:
    f.write(str(model.inputs[0].name) + " " + str(model.outputs[0].name) + "\n")



inputs:  ['dense_14_input']
outputs:  ['dense_18/Sigmoid']
INFO:tensorflow:Froze 26 variables.
INFO:tensorflow:Converted 26 variables to const ops.


In [14]:

import os
import pickle

import yaml
yaml_vars = yaml.safe_load(open(os.path.join(model_dir, "variables.yml"), "r"))

scaler = pickle.load(open(os.path.join(model_dir, "scaler_model.pkl"), 'rb'))
with open(os.path.join(model_dir, "output_scaler.txt"), "w") as f:
    for var, mean, scale in zip(yaml_vars, scaler.mean_, scaler.scale_):
        f.write(var + " " + str(mean) + " " + str(scale) + "\n")
